## Resources

Use this github as a starter to getting the code for numerapi: https://github.com/uuazed/numerapi

# Model Training

In [8]:
import os
import joblib
import pandas as pd
import numpy as np
import numerapi as nai
from zipfile import ZipFile
import joblib
from xgboost import XGBRegressor
import re
from sklearn.linear_model import LinearRegression()
from sklearn.model_selection import train_test_split
napi = nai.NumerAPI(verbosity='info')
x = napi.download_current_dataset()
# save predictions to project space
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='INSERT_PROJECT_ID_HERE', project_access_token='INSERT_PROJECT_ACCESS_TOKEN_HERE')
pc = project.project_context

2022-06-07 15:16:27,984 - __PROJECT_LIB__ - INFO - GET https://api.dataplatform.cloud.ibm.com/v2/projects/37382622-adc4-4b49-9689-55f2f31ece47
2022-06-07 15:16:27,984 INFO __PROJECT_LIB__: GET https://api.dataplatform.cloud.ibm.com/v2/projects/37382622-adc4-4b49-9689-55f2f31ece47
2022-06-07 15:16:28,082 - __PROJECT_LIB__ - INFO - No spark context provided, skipping setup of Hadoop config.
2022-06-07 15:16:28,082 INFO __PROJECT_LIB__: No spark context provided, skipping setup of Hadoop config.


In [6]:
with ZipFile(x, 'r') as zip_ref:
    zip_ref.extractall('path/to/directory')
train_df = pd.read_csv('path/to/directory/numerai_training_data.csv')
test_df = pd.read_csv('path/to/directory/numerai_tournament_data.csv')
model = LinearRegression()
featnames = [f for f in train_df if f.startswith("feature")]
X_train = train_df[featnames]
y_train = train_df['target']
model.fit(X_train, y_train)

2022-06-05 18:59:52,631 INFO numerapi.utils: target file already exists
2022-06-05 18:59:52,633 INFO numerapi.utils: download complete
2022-06-05 18:59:52,636 INFO numerapi.base_api: unzipping file...


LinearRegression()

In [7]:
# score model on tournament data
X_test = test_df[featnames]
y_test = test_df['target']
test_df['prediction'] = model.predict(X_test)
predictions = test_df[['id', 'prediction']]

# Upload Predictions

In [1]:
# un-comment the below line to save your submissions back to the project space
#project.save_data("submission.csv", predictions.to_csv(index=False), overwrite=True)

In [10]:
# use numerapi to upload predictions to the numerai tournament
# upload predictions
# provide api tokens
example_public_id = "ENTER-YOUR-PUBLIC-ID"
example_secret_key = "ENTER-YOUR-SECRET-KEY"
napi = nai.NumerAPI(example_public_id, example_secret_key)

In [11]:
napi.get_models()

{'dc_ai_': 'bea43b8f-3a2c-41ce-8d58-da477251c012',
 'jrdc': '1e2bd437-cc25-4226-832c-93cd3c61525f',
 'logreg': 'fdd25464-46bf-4247-b628-913a864cae94',
 'xgbr_1': '56a47781-5efc-4beb-b316-115de257d18c'}

In [12]:
model_id = napi.get_models()['xgbr_1']

In [13]:
napi.upload_predictions(df=predictions, model_id=model_id)

2022-06-05 19:01:57,128 INFO numerapi.base_api: uploading predictions...


'fcdfdc42-63c2-42d8-8989-a1a5e21adbaa'

In [14]:
# check submission status
print(napi.submission_status(model_id))

{'corrWithExamplePreds': None, 'filename': 'predictions-Hj1v70HRwrxa.csv', 'validationCorrPlusMmcSharpe': None, 'validationCorrPlusMmcSharpeDiff': None, 'validationCorrPlusMmcSharpeDiffRating': None, 'validationCorrPlusMmcSharpeRating': None, 'validationCorrelation': None, 'validationCorrelationRating': None, 'validationFeatureExposure': None, 'validationFeatureNeutralMean': None, 'validationFeatureNeutralMeanRating': None, 'validationMaxDrawdown': None, 'validationMaxDrawdownRating': None, 'validationMaxFeatureExposure': None, 'validationMaxFeatureExposureRating': None, 'validationMmcMean': None, 'validationMmcMeanRating': None, 'validationSharpe': None, 'validationSharpeRating': None, 'validationStd': None, 'validationStdRating': None}


## After setting up model scoring and prediction uploads, just set up a notebook job from the UI for this to run every Weekend on Tournament Reset